In [1]:
import numpy as np
import sqlite3
import pandas as pd
import csv

from datetime import datetime, timedelta
import time
import datetime
import requests
from io import StringIO

In [2]:
# 抓上市公司股價
def stock_value(datestr):
    #把當月11號的台股資訊 更新
    #datestr = '20200911'
    # 下載股價
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
    # 整理資料，變成表格
    df = pd.read_csv(StringIO(r.text.replace("=", "")), 
                header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    # 整理一些字串：
    df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
    # 顯示出來
    #df.head()
    return df

In [3]:
#抓上櫃公司股價
def stock_value2(datestr):
    link = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d='+datestr+'&s=0,asc,0'
    r = requests.get(link)
    r.ok
    lines = r.text.replace('\r', '').split('\n')
    df = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
    #df.head()
    df.columns = list(map(lambda l: l.replace(' ',''), lines[2].split(',')))
    #df.index = df['代號']
    #df = df.drop(['代號'], axis=1)
    #df.head()
    return df


In [5]:
nowyear = input('請輸入起始的年份：')
nowmonth = input('請輸入起始的月份：')

 #往前推一個月
if int(nowmonth)==1:
    last_month=str(12)
    last_year=str(int(nowyear)-1)
else:
    last_month=str(int(nowmonth)-1)  
    last_year=nowyear
if len(last_month)==1:
    last_month="0"+last_month
else:
    pass

print("往前一個月",last_year,last_month)


#往前推三個月
if int(nowmonth)==1 or int(nowmonth)==2 or int(nowmonth)==3  :
    if int(nowmonth)==1:
        last_3month=str(10)
    elif int(nowmonth)==2:
        last_3month=str(11)
    elif int(nowmonth)==3:    
        last_3month=str(12)
    last3_year=str(int(nowyear)-1)
else:
    last_3month=str(int(nowmonth)-3)  
    last3_year=nowyear
if len(last_3month)==1:
    last_3month="0"+last_3month
else:
    pass

print("往前三個月",last3_year,last_3month)


 #往前推一年
last_year_month=nowmonth
last_year_year=str(int(nowyear)-1)
print("往前一年",last_year_year,last_year_month)

請輸入起始的年份：2022
請輸入起始的月份：09
往前一個月 2022 08
往前三個月 2022 06
往前一年 2021 09


In [6]:
## 拿上市收盤價資料
def take_value(df,id):
    take_info=df[pd.to_numeric(df['證券代號'], errors='coerce') == id]
    value=float(take_info["收盤價"].values)
    return value
#拿上櫃收盤價資料
def take_value2(df,id):
    take_info=df[pd.to_numeric(df['代號'], errors='coerce') == id]
    value=float(take_info["收盤"].values)
    return value

In [7]:
import time

#現在的價錢
a=nowyear+nowmonth+"11"
for i in range(100):
    try:
        df_now=stock_value(a)
        break
    except:
        from datetime import datetime
        edit_time=datetime.strptime(a, "%Y%m%d")
        import datetime
        a= (edit_time+datetime.timedelta(days=-1)).strftime("%Y%m%d")
        
print("sleep 1.5 seconds.")
time.sleep(5)
print("printed after 1.5 seconds.")




b=last_year+last_month+'11'
for i in range(100):
    try:
        df_last_month=stock_value(b)
        break
    except:
        from datetime import datetime
        edit_time=datetime.strptime(b, "%Y%m%d")
        import datetime
        b= (edit_time+datetime.timedelta(days=-1)).strftime("%Y%m%d")
    
print("sleep 1.5 seconds.")
time.sleep(5)
print("printed after 1.5 seconds.")



c=last3_year+last_3month+'11'
for i in range(100):
    try:
        df_last_3month=stock_value(c)
        break
    except:
        from datetime import datetime
        edit_time=datetime.strptime(c, "%Y%m%d")
        import datetime
        c= (edit_time+datetime.timedelta(days=-1)).strftime("%Y%m%d")

print("sleep 1.5 seconds.")
time.sleep(5)
print("printed after 1.5 seconds.")


d=last_year_year+last_year_month+'11'

for i in range(100):
    try:
        df_last_year=stock_value(d)
        break
    except:
        from datetime import datetime
        edit_time=datetime.strptime(d, "%Y%m%d")
        import datetime
        d= (edit_time+datetime.timedelta(days=-1)).strftime("%Y%m%d")
        

sleep 1.5 seconds.
printed after 1.5 seconds.
sleep 1.5 seconds.
printed after 1.5 seconds.
sleep 1.5 seconds.
printed after 1.5 seconds.


In [8]:
#0050的價格  現在的   一個月前  三個月前    一年前
now_price=take_value(df_now,50)
last_month_price=take_value(df_last_month,50)
df_last_3month_price=take_value(df_last_3month,50)
df_last_year_price=take_value(df_last_year,50)

In [9]:
monthly_0050_return=(now_price-last_month_price)/last_month_price
season_0050_return=(now_price-df_last_3month_price)/df_last_3month_price
year_0050_return=(now_price-df_last_year_price)/df_last_year_price

In [10]:
monthly_0050_return=round(monthly_0050_return*100,2)
season_0050_return=round(season_0050_return*100,2)
year_0050_return=round(year_0050_return*100,2)

In [11]:
#開始計算左邊策略當月的報酬跟震幅
db =  sqlite3.connect('db.sqlite3')
cursor = db.execute("SELECT * from basicCurrent")
row=cursor.fetchall()

fundamental_return=0
fundamental_amplitude=0

start_price_total=0
final_price_total=0

for i in range(len(row)):

    start_price_total=start_price_total+float(row[i][4])
    final_price_total=final_price_total+float(row[i][6])
        
    if abs(float(row[i][7]))>abs(float(fundamental_amplitude)):
        fundamental_amplitude=abs(float(row[i][7]))
        
fundamental_return=(float(final_price_total)-float(start_price_total))/float(start_price_total)
fundamental_return=round(fundamental_return*100,2)


#開始計算右邊策略當月的報酬跟震幅
db =  sqlite3.connect('db.sqlite3')
cursor = db.execute("SELECT * from technicCurrent")
row=cursor.fetchall()

technology_return=0
technology_amplitude=0

start_price_total=0
final_price_total=0

for i in range(len(row)):

    start_price_total=start_price_total+float(row[i][4])
    final_price_total=final_price_total+float(row[i][6])
        
    if abs(float(row[i][7]))>abs(float(technology_amplitude)):
        technology_amplitude=abs(float(row[i][7]))
        
technology_return=(float(final_price_total)-float(start_price_total))/float(start_price_total)
technology_return=round(technology_return*100,2)

In [12]:
#開始計算策略的報酬   #最上面的區塊
db =  sqlite3.connect('db.sqlite3')
cursor = db.execute("SELECT * from basicHistory")
row=cursor.fetchall()

In [13]:
#開始計算策略的報酬
db =  sqlite3.connect('db.sqlite3')
cursor = db.execute("SELECT * from technicHistory")
row=cursor.fetchall()

In [14]:
a=nowyear+"-"+nowmonth+"-"+"11"
b=last_year+"-"+last_month+"-"+'11'
c=last3_year+"-"+last_3month+"-"+'11'
d=last_year_year+"-"+last_year_month+"-"+'11'

In [15]:
#計算上個月的績效
start_price_total=0
final_price_total=0
monthly_return=0
for i in range(len(row)-1,0,-1):
    if row[i][2][:7]==b[:7]:
        #print(row[i])
        start_price_total=start_price_total+float(row[i][3])
        final_price_total=final_price_total+float(row[i][5])
monthly_return=(float(final_price_total)-float(start_price_total))/float(start_price_total)
monthly_return

0.003980137219968917

In [16]:
    for i in range(len(row)-1,0,-1):
        print(row[i][2][0:10],b)
        #print(i)
        if row[i][2].replace("\n","")[:7]==b[:7]:
            print(row[i],row[i][3],row[i][5])
            start_price_total2=start_price_total2+float(row[i][3])
            final_price_total2=final_price_total2+float(row[i][5])

2022-08-11 2022-08-11
(3915, '9945 潤泰新', '2022-08-11', '64.5', '2022-9-11', '60.3', '-6.51', '-') 64.5 60.3


NameError: name 'start_price_total2' is not defined

In [ ]:
row

In [ ]:
#計算上一季的績效

season_return=1
last_month=nowmonth
for j in range(1,4,1):
    start_price_total2=0
    final_price_total2=0
    #往前推一個月
    if int(last_month)==1:
        last_month=str(12)
        last_year=str(int(last_year)-1)
    else:
        last_month=str(int(last_month)-1)  

    if len(last_month)==1:
        last_month="0"+last_month
    else:
        pass
    
    b=last_year+"-"+last_month+"-"+'11'  
    #print(last_month)
    
    for i in range(len(row)-1,0,-1):
        #print(row[i][2][0:10],b)
        #print(i)
        if row[i][2].replace("\n","")[:7]==b[:7]:
            print(row[i],row[i][3],row[i][5])
            start_price_total2=start_price_total2+float(row[i][3])
            final_price_total2=final_price_total2+float(row[i][5])
    #print("-------")   
    temp=(float(final_price_total2)-float(start_price_total2))/float(start_price_total2)
    season_return=season_return*(1+temp)

    #print(temp)
if season_return>1:
    season_return=season_return-1

    season_return

In [ ]:
float(start_price_total2)

In [ ]:
last_year="2021"

In [ ]:
#計算年的績效
year_return=1
last_month=nowmonth
for j in range(1,13,1):
    start_price_total3=0
    final_price_total3=0
    #往前推一個月
    if int(last_month)==1:
        last_month=str(12)
        last_year=str(int(last_year)-1)
    else:
        last_month=str(int(last_month)-1)  

    if len(last_month)==1:
        last_month="0"+last_month
    else:
        pass
    
    b=last_year+"-"+last_month+"-"+'11'  
    print(last_month)
    
    for i in range(len(row)-1,0,-1):
        #print(row[i][2][0:10],b)
        if row[i][2].replace("\n","")==b:
            #print(row[i])
            start_price_total3=start_price_total3+float(row[i][3])
            final_price_total3=final_price_total3+float(row[i][5])
    print("-------")   
    try:
        temp=(float(final_price_total3)-float(start_price_total3))/float(start_price_total3)
    except:
        temp=0
        print(b)
    year_return=year_return*(1+temp)
    print("year_return",year_return)
    print(temp)  #每個月的績效
if year_return>1:
    year_return=year_return-1
year_return

In [ ]:
monthly_return,(season_return),(year_return)

In [ ]:
#monthly_return=round(((fundamental_return+technology_return)/2),2)

monthly_return=round(((monthly_return)*100),2)
season_return=round(((season_return)*100),2)
year_return=round(((year_return)*100),2)

In [ ]:
# 更新bot database
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from bot")
db.commit()
pk=1

# final_update,name,start_date,start_price,over_date,current_price,now_return,types=change_parameter(now_list[i]) 
db =  sqlite3.connect('db.sqlite3')
db.execute("INSERT INTO bot (id,monthly_return,monthly_0050_return,season_return,season_0050_return,year_return,year_0050_return,fundamental_return,fundamental_amplitude,technology_return,technology_amplitude)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,monthly_return,monthly_0050_return,season_return,season_0050_return,year_return,year_0050_return,fundamental_return,fundamental_amplitude,technology_return,technology_amplitude))
db.commit()
db.close()


In [ ]:
#over

(-0.00498017912200826, 0.681349506366397, 0.8916452251307014)

In [15]:
#開始計算策略的報酬
db =  sqlite3.connect('db.sqlite3')
cursor = db.execute("SELECT * from technicHistory")
row=cursor.fetchall()

In [16]:
start_price_total=0
final_price_total=0

start_price_total2=0
final_price_total2=0

start_price_total3=0
final_price_total3=0



for i in range(len(row)):
    if b<row[i][2] or b==row[i][2]:
        start_price_total=start_price_total+float(row[i][3])
        final_price_total=final_price_total+float(row[i][5])
    if c<row[i][2] or c==row[i][2]:
        start_price_total2=start_price_total2+float(row[i][3])
        final_price_total2=final_price_total2+float(row[i][5])
    if d<row[i][2] or d==row[i][2]:
        start_price_total3=start_price_total3+float(row[i][3])
        final_price_total3=final_price_total3+float(row[i][5])
        

monthly_return=monthly_return+(float(final_price_total)-float(start_price_total))/float(start_price_total)
season_return=season_return+(float(final_price_total2)-float(start_price_total2))/float(start_price_total2)
year_return=year_return+(float(final_price_total3)-float(start_price_total3))/float(start_price_total3)

In [17]:
monthly_return,season_return,year_return

(0.020300299561611217, 0.054177651407478065, 0.09758573901833878)

In [18]:
#monthly_return=round(((fundamental_return+technology_return)/2),2)

monthly_return=round(((monthly_return)*100),2)
season_return=round(((season_return)*100),2)
year_return=round(((year_return)*100),2)

In [19]:
monthly_return,season_return,year_return

(2.03, 5.42, 9.76)